In [1]:
import pandas as pd
import numpy as np
import regex as re
import spacy
from pathlib import Path
from collections import defaultdict

files_path = Path("/home/hombre/Code/Climate Project/Classification/files")
#nlp = spacy.load('pl_core_news_sm')

2023-09-03 13:33:25.715235: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 13:33:25.761597: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 13:33:26.836873: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
stopwords = pd.read_csv(files_path / 'polish_stopwords.txt', header=None)
STOPWORDS = set([word.rstrip() for word in stopwords[0]])


REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;\.]')
REPLACE_EXCESS_SPACES_RE = re.compile('\s\s+')
BAD_SYMBOLS_RE = re.compile('[^AaĄąBbCcĆćDdEeĘęFfGgHhIiJjKkLlŁłMmNnŃńOoÓóPpRrSsŚśTtUuWwYyZzŹźŻż1234567890 ]')

def lemmatizer(text):
  return 
  doc = nlp(text)
  return " ".join([token.lemma_ for token in doc])

def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = REPLACE_EXCESS_SPACES_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # No Stemming
    #text = ' '.join(stemmer.stem(word) for word in text.split() if word not in STOPWORDS and stemmer.stem(word) != None) # Stemming
    if text == None:
      text == "None None None"
    return text

def preprocess(df):
    df['title'] = df['title'].fillna("None")
    df['clean_text'] = df['text'].apply(clean_text)
    df['clean_title'] = df['title'].apply(clean_text)
    df['clean_title'] = df['clean_title'].apply(lemmatizer)
    df['clean_text'] = df['clean_text'].apply(lemmatizer)
    return df


def generate_ngrams(text, n_gram=1):
    token = [token for token in text.lower().split(' ') if token != '' if token not in STOPWORDS]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [' '.join(ngram) for ngram in ngrams]


In [5]:
weak_grams= pd.read_csv("files/ngram/mis_ngrams.csv")

In [3]:
df_results = pd.read_csv(files_path / "rzepa"/ "rare.csv")

#df_results = df_results[df_results['label'] == 1]

In [4]:
def ngram_counter(ngram, df):
  eco_ngrams = defaultdict(int)

  for article in df['clean_text']:
      for word in generate_ngrams(article, ngram):
          eco_ngrams[word] += 1
        
          
  df_eco_ngrams = pd.DataFrame(sorted(eco_ngrams.items(), key=lambda x: x[1])[::-1])

  return df_eco_ngrams

In [5]:
df_eco_unigrams = ngram_counter(1, df_results)
df_eco_bigrams = ngram_counter(2, df_results)
df_eco_trigrams = ngram_counter(3, df_results)

In [6]:
df_1 =pd.read_csv(files_path/ "ngram/unigrams.csv")
df_2 =pd.read_csv(files_path/ "ngram/bigrams.csv")
df_3 =pd.read_csv(files_path/ "ngram/trigrams.csv")

df_eco = pd.concat([df_1, df_2, df_3])

eco_dict = df_eco.set_index("phrase").to_dict()['count']
eco_vocab = dict(zip(list(eco_dict.keys()), range(len(eco_dict.keys()))))

In [8]:
df_eco_unigrams[~df_eco_unigrams[0].isin(eco_vocab.keys())].iloc[:400].to_csv(files_path/ "ngram/uni_rest.csv", index = False)
df_eco_bigrams[~df_eco_bigrams[0].isin(eco_vocab.keys())].iloc[:400].to_csv(files_path/ "ngram/bi_rest.csv", index = False)
df_eco_trigrams[~df_eco_trigrams[0].isin(eco_vocab.keys())].iloc[:400].to_csv(files_path/ "ngram/tri_rest.csv", index = False)